In [106]:
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
import os


os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

graph = Neo4jGraph(
    url="bolt://localhost:7687", 
    username="neo4j", 
    password="123456789",
    enhanced_schema=True
)

os.environ['OPENAI_API_KEY'] = "GPT key"
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(
    llm=llm, graph=graph, verbose=True,
     allow_dangerous_requests=True
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [123]:
graph.refresh_schema()
print(graph.schema)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


Node properties:
- **Developer**
  - `availability`: INTEGER Min: 3, Max: 5
  - `code_review_frequent`: INTEGER Min: 4, Max: 4
  - `id`: STRING Available options: ['quang', 'sang', 'hani', 'duong', 'nhi']
  - `name`: STRING Available options: ['Quang', 'Sang', 'Hani', 'Duong', 'Nhi']
  - `responsiveness`: INTEGER Min: 2, Max: 5
  - `review_score`: FLOAT Min: 50.0, Max: 99.0
- **Skill**
  - `id`: STRING Available options: ['js', 'py', 'reactjs', 'ts', 'cicd', 'platform', 'ruby']
  - `name`: STRING Available options: ['JavaScript', 'Python', 'ReactJS', 'Typescript', 'CI/CD', 'Platform', 'Ruby']
- **Team**
  - `id`: STRING Available options: ['helios', 'konoha', 'web-platform']
  - `name`: STRING Available options: ['Helios', 'Konoha', 'Web Platform']
  - `domains`: LIST Min Size: 1, Max Size: 2
- **PR**
  - `id`: STRING Available options: ['PR1', 'PR2']
  - `domains`: LIST Min Size: 1, Max Size: 1
  - `description`: STRING Available options: ['Fixes a critical bug to ATS', 'Adds a new fe

In [107]:
from operator import add
from typing import Annotated, List

from typing_extensions import TypedDict


class InputState(TypedDict):
    question: str


class OverallState(TypedDict):
    question: str
    next_action: str
    cypher_statement: str
    cypher_errors: List[str]
    database_records: List[dict]
    steps: Annotated[List[str], add]


class OutputState(TypedDict):
    answer: str
    steps: List[str]
    cypher_statement: str

In [108]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

guardrails_system = """
As an intelligent assistant, your primary objective is to decide whether a given question is related to code contribution or not. 
If the question is related to code contribution, output "code contribution". Otherwise, output "end".
To make this decision, assess the cntent of the question and determine if it refers to any developers, teams, skills, pull requests, code reviews, contributions, mentoring,...
or related topics. Provide only the specified output: "code contribution" or "end".
"""
guardrails_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            guardrails_system,
        ),
        (
            "human",
            ("{question}"),
        ),
    ]
)


class GuardrailsOutput(BaseModel):
    decision: Literal["code contribution", "end"] = Field(
        description="Decision on whether the question is related to code contribution"
    )


guardrails_chain = guardrails_prompt | llm.with_structured_output(GuardrailsOutput)


def guardrails(state: InputState) -> OverallState:
    """
    Decides if the question is related to code contribution or not.
    """
    guardrails_output = guardrails_chain.invoke({"question": state.get("question")})
    database_records = None
    if guardrails_output.decision == "end":
        database_records = "This questions is not about code contributions or related info. Therefore I cannot answer this question."
    return {
        "next_action": guardrails_output.decision,
        "database_records": database_records,
        "steps": ["guardrail"],
    }

In [109]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_neo4j import Neo4jVector
from langchain_openai import OpenAIEmbeddings

examples = [
    {
        "question": "Which developers have the highest level of skill in ReactJS?",
        "query": """
          MATCH (d:Developer)-[r:DEVELOPER_HAS_SKILL]->(s:Skill)
          WHERE s.name = "ReactJS"
          RETURN d.name AS Developer, r.level AS SkillLevel
          ORDER BY r.level DESC
          LIMIT 5
        """,
    },
    {
        "question": "Which team own file1.js?",
        "query": """
          MATCH (t:Team)-[:CODEOWNER_OF]->(f:File {name: "file1.js"})
          RETURN t.name AS Team, f.name AS File, f.domain AS Domain
        """,
    },
    {
        "question": "Which pull request affected file file1.js?",
        "query": """
          MATCH (pr:PR)-[:PR_AFFECTS_FILE]->(f:File {name: "file1.js"})
          RETURN pr.title AS PullRequest, pr.description AS Description, pr.timestamp AS Timestamp
        """,
    },
    {
        "question": "Which developer has the most contribution in ATS domain? Contributions includes code contributions and reviews.",
        "query": """
          MATCH (d:Developer)
          OPTIONAL MATCH (d)-[:DEVELOPER_CONTRIBUTED_TO]->(pr:PR)
          WHERE "ATS" IN pr.domains
          WITH d, COUNT(pr) AS CodeContributions
          OPTIONAL MATCH (d)-[:DEVELOPER_REVIEWED]->(pr:PR)
          WHERE "ATS" IN pr.domains
          WITH d, CodeContributions, COUNT(pr) AS Reviews
          RETURN d.name AS Developer, (CodeContributions + Reviews) AS TotalContributions
          ORDER BY TotalContributions DESC
          LIMIT 1
        """,
    },
    {
        "question": "Which developers contributed to pull request 'PR1'?",
        "query": """
          MATCH (d:Developer)-[:DEVELOPER_CONTRIBUTED_TO]->(pr:PR {id: "PR1"})
          RETURN d.name AS Developer, pr.title AS PullRequest
        """,
    },
    {
        "question": "Which developers have a review score higher than 90?",
        "query": """
          MATCH (d:Developer)
          WHERE d.review_score > 90
          RETURN d.name AS Developer, d.review_score AS ReviewScore
          ORDER BY d.review_score DESC
        """,
    },
     {
        "question": "Which developers are members of the 'Konoha' team?",
        "query": """
          MATCH (d:Developer)-[:IS_MEMBER_OF_TEAM]->(t:Team {name: "Konoha"})
          RETURN d.name AS Developer, t.name AS Team
        """,
    },
    {
        "question": "Which developers have a skill level of 5 in JavaScript?",
        "query": """
          MATCH (d:Developer)-[r:DEVELOPER_HAS_SKILL]->(s:Skill)
          WHERE s.name = "JavaScript" AND r.level = 5
          RETURN d.name AS Developer, s.name AS Skill, r.level AS SkillLevel
        """,
    },
    {
        "question": "Find top 3 developers to review a PR created by Nhi, with tech stack Ruby, and domain ATS, SmartMatch file change: file4.rb and file2.py",
        "query": """
            WITH [
                "Ruby"
            ] AS techInput,
            ["ATS", "SmartMatch"] AS prDomains, 
            ["file4.rb", "file2.py"] AS fileChanges, 
            ["nhi"] AS prContributors

            MATCH (dev:Developer)
            WHERE NOT dev.id IN prContributors

            OPTIONAL MATCH (dev)-[hs:DEVELOPER_HAS_SKILL]->(skill:Skill)
            WHERE skill.name IN techInput
            WITH dev, prDomains, fileChanges,
                SUM(COALESCE(hs.level, 0)) AS actualSkillScore,
                COUNT(skill) AS matchedSkills,
                SIZE(techInput) * 5 AS maxSkillScore
            WITH dev, prDomains, fileChanges,
                (TOFLOAT(actualSkillScore) / COALESCE(maxSkillScore, 1)) AS skillMatchScore

            OPTIONAL MATCH (dev)-[contributed:DEVELOPER_CONTRIBUTED_TO]->(pr:PR)-[:PR_AFFECTS_FILE]->(file:File)
            WHERE file.name IN fileChanges
            WITH dev, prDomains, fileChanges, skillMatchScore,
                COUNT(DISTINCT CASE WHEN file IS NOT NULL THEN file END) AS contributionCount
            WITH dev, prDomains, fileChanges, skillMatchScore,
                CASE 
                    WHEN contributionCount IS NULL THEN 0 
                    WHEN contributionCount = 0 THEN 0 
                    WHEN contributionCount = 1 THEN 0.55 
                    WHEN contributionCount = 2 THEN 0.75 
                    ELSE 1.0 
                END AS workHistoryScore

            OPTIONAL MATCH (dev)-[reviewed:DEVELOPER_REVIEWED]->(pr:PR)-[:PR_AFFECTS_FILE]->(file:File)
            WHERE file.name IN fileChanges
            WITH dev, prDomains, fileChanges, skillMatchScore, workHistoryScore,
                COALESCE(AVG(reviewed.review_score), 0) AS reviewQualityScore

            WITH dev, prDomains, fileChanges, skillMatchScore, workHistoryScore, reviewQualityScore,
                dev.code_review_frequent * 0.2 AS reviewFrequencyScore,
                dev.availability * 0.2 AS availabilityScore

            OPTIONAL MATCH (dev)-[:IS_MEMBER_OF_TEAM|LEADER_OF]->(team:Team)
            WITH dev, prDomains, skillMatchScore, workHistoryScore, reviewQualityScore, 
                reviewFrequencyScore, availabilityScore,
                COUNT(DISTINCT CASE WHEN ANY(domain IN team.domains WHERE domain IN prDomains) THEN 1 END) AS domainMatches,
                SIZE(prDomains) AS totalDomains
            WITH dev, skillMatchScore, workHistoryScore, reviewQualityScore, 
                reviewFrequencyScore, availabilityScore,
                (TOFLOAT(domainMatches) / COALESCE(totalDomains, 1)) AS domainKnowledgeScore

            OPTIONAL MATCH (dev)-[:MENTOR]->(mentee:Developer)
            WITH dev, skillMatchScore, workHistoryScore, reviewQualityScore, 
                reviewFrequencyScore, availabilityScore, domainKnowledgeScore,
                CASE WHEN mentee IS NOT NULL THEN 1.0 ELSE 0.5 END AS orgPriorityScore

            WITH dev,
                (skillMatchScore * 0.2) +
                (workHistoryScore * 0.2) +
                (reviewQualityScore * 0.2) +
                (reviewFrequencyScore * 0.1) +
                (availabilityScore * 0.1) +
                (domainKnowledgeScore * 0.1) +
                (orgPriorityScore * 0.05) AS totalScore,
                skillMatchScore, workHistoryScore, reviewQualityScore, 
                reviewFrequencyScore, availabilityScore, domainKnowledgeScore

            RETURN dev.id AS developer, dev.name AS name, totalScore, skillMatchScore, workHistoryScore, reviewQualityScore, 
                reviewFrequencyScore, availabilityScore, domainKnowledgeScore
            ORDER BY totalScore DESC
            LIMIT 3
        """,
    }
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, OpenAIEmbeddings(), Neo4jVector, k=4, input_keys=["question"]
)

In [110]:
from langchain_core.output_parsers import StrOutputParser

text2cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "Given an input question, convert it to a Cypher query. No pre-amble."
                "Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!"
            ),
        ),
        (
            "human",
            (
                """You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.
Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!
Here is the schema information
{schema}

Below are a number of examples of questions and their corresponding Cypher queries.

{fewshot_examples}

User input: {question}
Cypher query:"""
            ),
        ),
    ]
)

text2cypher_chain = text2cypher_prompt | llm | StrOutputParser()


def generate_cypher(state: OverallState) -> OverallState:
    """
    Generates a cypher statement based on the provided schema and user input
    """
    NL = "\n"
    fewshot_examples = (NL * 2).join(
        [
            f"Question: {el['question']}{NL}Cypher:{el['query']}"
            for el in example_selector.select_examples(
                {"question": state.get("question")}
            )
        ]
    )

    generated_cypher = text2cypher_chain.invoke(
        {
            "question": state.get("question"),
            "fewshot_examples": fewshot_examples,
            "schema": graph.schema,
        }
    )

    return {"cypher_statement": generated_cypher, "steps": ["generate_cypher"]}

In [111]:
from typing import List, Optional

validate_cypher_system = """
You are a Cypher expert reviewing a statement written by a junior developer.
"""

validate_cypher_user = """You must check the following:
* Are there any syntax errors in the Cypher statement?
* Are there any missing or undefined variables in the Cypher statement?
* Are any node labels missing from the schema?
* Ignore any missing relationship types and properties.
* Ignore some inoptimal points in the Cypher statement.
* Just need to make sure the query is syntactically correct.

Schema:
{schema}

The question is:
{question}

The Cypher statement is:
{cypher}

Make sure you don't make any mistakes!"""

validate_cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            validate_cypher_system,
        ),
        (
            "human",
            (validate_cypher_user),
        ),
    ]
)


class Property(BaseModel):
    """
    Represents a filter condition based on a specific node property in a graph in a Cypher statement.
    """

    node_label: str = Field(
        description="The label of the node to which this property belongs."
    )
    property_key: str = Field(description="The key of the property being filtered.")
    property_value: str = Field(
        description="The value that the property is being matched against."
    )


class ValidateCypherOutput(BaseModel):
    """
    Represents the validation result of a Cypher query's output,
    including any errors and applied filters.
    """

    errors: Optional[List[str]] = Field(
        description="A list of syntax or semantical errors in the Cypher statement. Always explain the discrepancy between schema and Cypher statement"
    )
    filters: Optional[List[Property]] = Field(
        description="A list of property-based filters applied in the Cypher statement."
    )


validate_cypher_chain = validate_cypher_prompt | llm.with_structured_output(
    ValidateCypherOutput
)

In [112]:
from langchain_neo4j.chains.graph_qa.cypher_utils import CypherQueryCorrector, Schema

# Cypher query corrector is experimental
corrector_schema = [
    Schema(el["start"], el["type"], el["end"])
    for el in graph.structured_schema.get("relationships")
]
cypher_query_corrector = CypherQueryCorrector(corrector_schema)

In [113]:
from neo4j.exceptions import CypherSyntaxError


def validate_cypher(state: OverallState) -> OverallState:
    """
    Validates the Cypher statements and maps any property values to the database.
    """
    # errors = []
    # mapping_errors = []
    # # Check for syntax errors
    # try:
    #     graph.query(f"EXPLAIN {state.get('cypher_statement')}")
    # except CypherSyntaxError as e:
    #     errors.append(e.message)
    # # Experimental feature for correcting relationship directions
    # corrected_cypher = cypher_query_corrector(state.get("cypher_statement"))
    # if not corrected_cypher:
    #     errors.append("The generated Cypher statement doesn't fit the graph schema")
    # if not corrected_cypher == state.get("cypher_statement"):
    #     print("Relationship direction was corrected")
    # # Use LLM to find additional potential errors and get the mapping for values
    # print(validate_cypher_chain, 'validate_cypher_chain')
    # llm_output = validate_cypher_chain.invoke(
    #     {
    #         "question": state.get("question"),
    #         "schema": graph.schema,
    #         "cypher": state.get("cypher_statement"),
    #     }
    # )
    # print(llm_output, 'llm_output')
    # if llm_output.errors:
    #     errors.extend(llm_output.errors)
    # if llm_output.filters:
    #     for filter in llm_output.filters:
    #         # Do mapping only for string values
    #         if (
    #             not [
    #                 prop
    #                 for prop in graph.structured_schema["node_props"][
    #                     filter.node_label
    #                 ]
    #                 if prop["property"] == filter.property_key
    #             ][0]["type"]
    #             == "STRING"
    #         ):
    #             continue
    #         mapping = graph.query(
    #             f"MATCH (n:{filter.node_label}) WHERE toLower(n.`{filter.property_key}`) = toLower($value) RETURN 'yes' LIMIT 1",
    #             {"value": filter.property_value},
    #         )
    #         if not mapping:
    #             print(
    #                 f"Missing value mapping for {filter.node_label} on property {filter.property_key} with value {filter.property_value}"
    #             )
    #             mapping_errors.append(
    #                 f"Missing value mapping for {filter.node_label} on property {filter.property_key} with value {filter.property_value}"
    #             )
    # if mapping_errors:
    #     next_action = "end"
    # elif errors:
    #     next_action = "correct_cypher"
    # else:
    #     next_action = "execute_cypher"

    return {
        "next_action": "execute_cypher",
        "cypher_statement": state.get("cypher_statement"),
        "cypher_errors": [],
        "steps": ["validate_cypher"],
    }

In [114]:
correct_cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a Cypher expert reviewing a statement written by a junior developer. "
                "You need to correct the Cypher statement based on the provided errors. No pre-amble."
                "Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!"
            ),
        ),
        (
            "human",
            (
                """Check for invalid syntax or semantics and return a corrected Cypher statement.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not wrap the response in any backticks or anything else.
Respond with a Cypher statement only!

Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.

The question is:
{question}

The Cypher statement is:
{cypher}

The errors are:
{errors}

Corrected Cypher statement: """
            ),
        ),
    ]
)

correct_cypher_chain = correct_cypher_prompt | llm | StrOutputParser()


def correct_cypher(state: OverallState) -> OverallState:
    """
    Correct the Cypher statement based on the provided errors.
    """
    corrected_cypher = correct_cypher_chain.invoke(
        {
            "question": state.get("question"),
            "errors": state.get("cypher_errors"),
            "cypher": state.get("cypher_statement"),
            "schema": graph.schema,
        }
    )

    return {
        "next_action": "validate_cypher",
        "cypher_statement": corrected_cypher,
        "steps": ["correct_cypher"],
    }

In [115]:
no_results = "I couldn't find any relevant information in the database"


def execute_cypher(state: OverallState) -> OverallState:
    """
    Executes the given Cypher statement.
    """

    records = graph.query(state.get("cypher_statement"))

    return {
        "database_records": records if records else no_results,
        "next_action": "end",
        "steps": ["execute_cypher"],
    }

In [116]:
generate_final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        (
            "human",
            (
                """Use the following results retrieved from a database to provide
a succinct, definitive answer to the user's question.

Respond as if you are answering the question directly.

Results: {results}
Question: {question}"""
            ),
        ),
    ]
)

generate_final_chain = generate_final_prompt | llm | StrOutputParser()


def generate_final_answer(state: OverallState) -> OutputState:
    """
    Decides if the question is related to code contribution.
    """
    final_answer = generate_final_chain.invoke(
        {"question": state.get("question"), "results": state.get("database_records")}
    )
    return {"answer": final_answer, "steps": ["generate_final_answer"]}

In [117]:
def guardrails_condition(
    state: OverallState,
) -> Literal["generate_cypher", "generate_final_answer"]:
    if state.get("next_action") == "end":
        return "generate_final_answer"
    elif state.get("next_action") == "code contribution":
        return "generate_cypher"


def validate_cypher_condition(
    state: OverallState,
) -> Literal["generate_final_answer", "correct_cypher", "execute_cypher"]:
    if state.get("next_action") == "end":
        return "generate_final_answer"
    elif state.get("next_action") == "correct_cypher":
        return "correct_cypher"
    elif state.get("next_action") == "execute_cypher":
        return "execute_cypher"

In [118]:
from IPython.display import Image, display
from langgraph.graph import END, START, StateGraph

langgraph = StateGraph(OverallState, input=InputState, output=OutputState)
langgraph.add_node(guardrails)
langgraph.add_node(generate_cypher)
langgraph.add_node(validate_cypher)
langgraph.add_node(correct_cypher)
langgraph.add_node(execute_cypher)
langgraph.add_node(generate_final_answer)

langgraph.add_edge(START, "guardrails")
langgraph.add_conditional_edges(
    "guardrails",
    guardrails_condition,
)
langgraph.add_edge("generate_cypher", "validate_cypher")
langgraph.add_conditional_edges(
    "validate_cypher",
    validate_cypher_condition,
)
langgraph.add_edge("execute_cypher", "generate_final_answer")
langgraph.add_edge("correct_cypher", "validate_cypher")
langgraph.add_edge("generate_final_answer", END)

langgraph = langgraph.compile()


In [122]:
langgraph.invoke({"question": "Find 3 developers to review a PR created by Nhi, with tech stack Ruby, and domain ATS, SmartMatch file change: file4.rb and file2.py"})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}